In [ ]:
import pathlib
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pathlib
from GPy.models import GPRegression
import GPy
import pickle
import matplotlib.colors as mcolors
from sklearn.neighbors import KernelDensity
from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.acquisitions import IntegratedVarianceReduction
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.core.acquisition.acquisition_per_cost import acquisition_per_expected_cost
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import KernelDensity
from emukit.core.interfaces.models import IModel
from emukit.experimental_design import ExperimentalDesignLoop
from sklearn.neighbors import KernelDensity
from src.ice_sheet import ice_sheet_runner
import random
from src.utils import load_data, prep_data
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import seaborn as sns
plt.style.use('seaborn-whitegrid')
sns.set_palette('colorblind')
mpl.rcParams['font.family'] = 'Arial'

In [ ]:
# Load data 
data = load_data([0,176])
sample_size = 3000 # TEST SIZE, if you want real plots s=3000

In [ ]:
# Table 1
def create_latex_table(data, column_headers, row_headers):
    table = "\\begin{tabular}{|c|" + "|c" * len(data[0]) + "|}\n"
    
    # Add column headers row
    table += "  \\hline\n"
    table += "  & " + " & ".join(column_headers) + " \\\\ \n"
    table += "  \\hline\n"
    
    # Add table data
    for i, row in enumerate(data):
        table += "  " + row_headers[i] + " & " + " & ".join(row) + " \\\\ \n"
        table += "  \\hline\n"
    
    # Close the table
    table += "\\end{tabular}"
    
    return table

# Additional column headers
column_headers = ["2km", "3km", "4km", "5km", "6km", "7km", "8km"]

# Additional row headers
row_headers = ["100 years", "125 years", "150 years", "175 years"]


res_list = range(2,9)
results_list = []
for predictions in prediction_per_timestamp:
    results = []
    for idx,(r,i) in enumerate(zip(res_list,predictions)):
        mean,var = i[1]
        data_list = []
        for m,v in zip(mean,var):
            data_list += list(np.random.normal(m,np.sqrt(v),size=sample_size))
        
        # Instantiate and fit the KDE model
        kde = KernelDensity(kernel='gaussian', bandwidth=1.0)
        kde.fit(np.array(data_list).reshape((-1,1)))
        # Sample a large number of points from the fitted density
        n_samples = 100000
        samples = kde.sample(n_samples)
        # Calculate the proportion of points greater than 95% of the 2km threhshold
        if idx == 0:
            threshold = np.percentile(samples, 95)
        tail_prob = np.sum(samples > threshold) / n_samples
        print(f'Estimated probability of tail event: {tail_prob} for resolution {r}, with threshold {threshold}')

        results.append(tail_prob)
    results_list.append(results)

latex_table = create_latex_table([[str(round((p*100),1)) for p in i]for i in results_list], column_headers, row_headers)
print(latex_table)